# **Cosmetics E-Commerce Funnel Analysis**  

* Data Source: https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop

In [22]:
import os
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('cosmetic.csv')
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3533286 entries, 0 to 3533285
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 242.6+ MB


In [10]:
df = df.dropna(subset=['user_session'])

df['category_code'] = df['category_code'].fillna('unknown')
df['event_time'] = pd.to_datetime(df['event_time'])
df['event_type'] = df['event_type'].astype('category')
df['user_id'] = df['user_id'].astype(str) 

df = df.drop_duplicates()

## Funnel Construction

In [12]:
# pivot per session
session_funnel = df.pivot_table(index='user_session',
                                columns='event_type',
                                values='event_time',
                                aggfunc='count').fillna(0)

In [13]:
# convert each funnel to boolean
session_funnel['viewed'] = session_funnel['view'] > 0
session_funnel['carted'] = session_funnel['cart'] > 0
session_funnel['purchased'] = session_funnel['purchase'] > 0

In [14]:
total_sessions = len(session_funnel)

viewed = session_funnel['viewed'].sum()
carted = session_funnel[session_funnel['carted']]['viewed'].sum()
purchased = session_funnel[session_funnel['purchased']]['viewed'].sum()

In [15]:
# conversion rate calculation
view_to_cart = carted / viewed
cart_to_purchase = purchased / carted
view_to_purchase = purchased / viewed

print(f"Total sessions: {total_sessions}")
print(f"View → Cart Conversion Rate: {view_to_cart:.3f}")
print(f"Cart → Purchase Conversion Rate: {cart_to_purchase:.3f}")
print(f"View → Purchase Conversion Rate: {view_to_purchase:.3f}")

Total sessions: 839812
View → Cart Conversion Rate: 0.164
Cart → Purchase Conversion Rate: 0.167
View → Purchase Conversion Rate: 0.027


#### **Business Insights**
* The e-commerce funnel shows a sharp drop-off, with only 16.4% of views leading to cart actions and just 2.7% ending in purchase.
* The cart-to-purchase rate (16.7%) indicates significant hesitation at the final conversion step, suggesting issues like pricing, friction, or trust.
* Immediate opportunities exist in reducing cart abandonment and improving product-to-cart engagement via personalized nudges or UX optimization.

## Funnel Drop-off Analysis

In [16]:
# to cart, but no purchase
cart_drop_sessions = session_funnel[(session_funnel['carted']) & (~session_funnel['purchased'])]

drop_ratio = len(cart_drop_sessions) / session_funnel['carted'].sum()
print(f"Cart → Purchase Drop-off Rate: {drop_ratio:.2%}")

Cart → Purchase Drop-off Rate: 85.72%


In [17]:
remove_df = df[df['event_type'] == 'remove_from_cart']

remove_df['price'].describe()

count    496382.000000
mean          5.365182
std          10.271196
min           0.000000
25%           1.730000
50%           3.760000
75%           5.560000
max         327.780000
Name: price, dtype: float64

In [18]:
remove_df['brand'].value_counts().head(10)

brand
runail       37362
masura       24767
irisk        22213
grattol      20121
bpw.style    13265
ingarden     10791
pole          8419
estel         7897
bluesky       6530
freedecor     6254
Name: count, dtype: int64

#### **Business Insights**

* 85.7% of sessions that added items to cart did not convert, highlighting a critical friction at the purchase step.
* Cart removals are concentrated in low-to-mid price ranges (median $3.76), suggesting trust, urgency, or perceived value as core drop-off drivers.
* Brands like Runail and Masura show high cart removals, warranting deeper UX, pricing, or content audits for potential mismatches.

## Segment Drop-off Analysis

### cart -> purchase conversion rate per price range

In [19]:
cart_df = df[df['event_type'] == 'cart'].copy()
purchase_df = df[df['event_type'] == 'purchase'].copy()

In [20]:
cart_prices = cart_df.groupby('user_session')['price'].mean().reset_index(name='cart_price')
purchased_sessions = purchase_df['user_session'].unique()
cart_prices['purchased'] = cart_prices['user_session'].isin(purchased_sessions)

In [21]:
bins = [0, 2, 5, 10, 20, 50, 100, 500]
labels = ['0-2','2-5','5-10','10-20','20-50','50-100','100+']
cart_prices['price_range'] = pd.cut(cart_prices['cart_price'], bins=bins, labels=labels, right=False)

conversion_by_price = cart_prices.groupby('price_range')['purchased'].mean().reset_index()
print(conversion_by_price)

  price_range  purchased
0         0-2   0.110179
1         2-5   0.143442
2        5-10   0.144622
3       10-20   0.172578
4       20-50   0.179475
5      50-100   0.192205
6        100+   0.118421


#### **Business Insights**

* Conversion rates are highest for mid-range products ($10–100), peaking at 19.2%, while both low-end (<$2) and high-end (>$100) items underperform.
* The lower-than-expected conversion for ultra-cheap items suggests quality skepticism, not price sensitivity.
* This implies opportunity in repositioning or bundling low-end products and enhancing trust-building UX for high-value purchases.

## Actionable Strategy Recommendations

### **Funnel Drop-off Analysis**

Funnel analysis revealed significant friction at the conversion stage.

- **Cart → Purchase Drop-off Rate:** **85.72%**
- **Overall Conversion Rate (View → Purchase):** **2.7%**

> A majority of users add items to cart but do not complete the purchase, highlighting friction at the decision point. This suggests issues related to trust, perceived value, or UX.

---

### **Cart Removal Analysis**

Analyzed 490K+ cart removal events to understand drop-off patterns.

- **Median price of removed items:** \$3.76
- **Most removed brands:** Runail, Masura, Irisk, Grattol

> Cart removals are concentrated in low- to mid-priced products. While some high-priced items exist, most drop-offs occur in inexpensive items—implying quality concerns or impulse regret.

---

### **Conversion Rate by Price Range**

| Price Range | Conversion Rate |
|-------------|------------------|
| \$0–2        | 11.0%            |
| \$2–5        | 14.3%            |
| \$5–10       | 14.5%            |
| \$10–20      | 17.3%            |
| \$20–50      | 17.9%            |
| \$50–100     | **19.2%**        |
| \$100+       | 11.8%            |

> Conversion rates are highest for mid-priced items (\$10–100), while both ultra-low and high-end products suffer from lower performance. Users may be skeptical of quality at the low end and hesitant due to price at the high end.

---

### **Strategic Recommendations**

#### 1. Reduce Cart Abandonment
- Send targeted reminders for abandoned carts (especially in \$0–2 and \$100+ segments)
- Trigger messages for prolonged cart idle times

#### 2. Reposition Low-End Products
- Reinforce value perception through reviews and bundling
- Highlight quality assurance on product pages

#### 3. Improve High-End Trust Factors
- Display more product details and user testimonials
- Offer installment payments or first-time buyer discounts

#### 4. Targeted UX A/B Tests
- Run UI experiments on high-drop brands like Runail and Masura
- Optimize cart CTA and product detail layout

---

In [23]:
# Save csv for tableau

session_funnel_final = session_funnel[['viewed', 'carted', 'purchased']].copy()
session_with_price = cart_prices[['user_session', 'cart_price', 'price_range', 'purchased']]
final_df = session_funnel_final.merge(session_with_price, left_index=True, right_on='user_session', how='left')
final_df.to_csv("session_summary.csv", index=False)
